In [68]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import pandas as pd

In [2]:
GOOGLE_BASE_URL = 'https://cloud.google.com/products'

In [3]:
soup = BeautifulSoup(requests.get(GOOGLE_BASE_URL).text, 'html.parser')

In [ ]:
category_soups = soup.find_all('div', {'class': 'cloud-product-card'})

In [71]:
services = []

for cat_soup in category_soups:
    cat_icon = cat_soup.find('svg')
    cat_title = cat_soup.find('a').text.strip()
    if 'arrow_forward' in cat_title:
        cat_title = cat_title[:cat_title.index('arrow_forward') - 1]
    
    headlines = cat_soup.find_all('a', {'class': 'cloud-product-card__headline'})
    sub_headlines = cat_soup.find_all('div', {'class': 'cloud-product-card__sub-headline'})
    
    
    for i in range(len(headlines)):
        link = headlines[i]['href']
        service = {
            'category_name': cat_title,
            'name': headlines[i].text.strip(),
            'short_description': sub_headlines[i].text.strip(),
            'long_description': '',
            'link': headlines[i]['href'],
            'icon': ''
        }

        if urlparse(link).netloc == 'cloud.google.com':
            
            service_soup = BeautifulSoup(requests.get(link).text, 'html.parser')
            
            try:
                service['long_description'] = service_soup.find('meta', {'property': 'og:description'})['content'].strip()

                article_soup = service_soup.find('article', {'id': 'cloud-site'})
                try_free_button_soup = article_soup.find('a')
                icon_class = try_free_button_soup['class'][-1]
                service['icon'] = f"{GOOGLE_BASE_URL}/_static/images/cloud/products/logos/svg/{icon_class}-icon.svg"
            except:
                continue
        
        services.append(service)

In [72]:
google_services_df = pd.DataFrame(services)
google_services_df.head()

,category_name,icon,link,long_description,name,short_description
0,Compute,https://cloud.google.com/products/_static/imag...,https://cloud.google.com/compute/,Google Compute Engine offers high performance ...,Compute Engine,"Scalable, high performance VMs."
1,Compute,https://cloud.google.com/products/_static/imag...,https://cloud.google.com/appengine/,Google App Engine lets developers build scalab...,App Engine,Serverless application platform for apps and b...
2,Compute,https://cloud.google.com/products/_static/imag...,https://cloud.google.com/kubernetes-engine/,Google Kubernetes Engine is a powerful cluster...,Kubernetes Engine,Run containerized applications.
3,Compute,https://cloud.google.com/products/_static/imag...,https://cloud.google.com/gke-on-prem/,GKE On-Prem will provision and manage the unde...,GKE On-Prem,Make apps “cloud-ready” and move them to the c...
4,Compute,https://cloud.google.com/products/_static/imag...,https://cloud.google.com/functions/,Google Cloud Functions makes it easy for devel...,Cloud Functions,Event-driven serverless compute platform.


In [75]:
google_services_df.to_csv('../data/raw/google_services.csv', index=False)